In [38]:
import pandas
import numpy as np
import os
import shutil
from pathlib import Path
import json
import math
from PIL import Image, ImageDraw
import glob

import torch
import segmentation_models_pytorch as smp
import cv2
import torch
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import train_test_split


In [39]:
device = torch.device("mps" if torch.backends.mps.is_available() else
                      "cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")    

Using device: mps


In [40]:
SEED = 42

In [41]:
data_dir = Path().cwd().parent / "dataset"
if not data_dir.exists():
    raise FileNotFoundError(f"Data directory {data_dir} does not exist.")

images_dir = data_dir / "images"
if not images_dir.exists():
    raise FileNotFoundError(f"Images directory {images_dir} does not exist.")
labels_dir = data_dir / "labels"
if not labels_dir.exists():
    raise FileNotFoundError(f"Labels directory {labels_dir} does not exist.")
masks_dir = data_dir / "masks"
if not masks_dir.exists():
    raise FileNotFoundError(f"Masks directory {masks_dir} does not exist.")

# images_dir.mkdir(exist_ok=True)
# labels_dir.mkdir(exist_ok=True)
# masks_dir.mkdir(exist_ok=True)

print(f"Data directory: {data_dir}")
print(f"Images directory: {images_dir}")
print(f"Labels directory: {labels_dir}")

Data directory: /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset
Images directory: /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset/images
Labels directory: /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset/labels


In [42]:
all_labels = set()

for jp in labels_dir.glob("*.json"):
    with open(jp, "r") as f:
        anno = json.load(f)
    for shape in anno.get("shapes", []):
        lbl = shape.get("label", "")
        all_labels.add(lbl)

print("Found labels:", all_labels)

Found labels: {'Nerve', 'nerve'}


In [43]:
CLASS_MAP = {"background": 0, "nerve": 1}

In [44]:
image_shapes = []
mask_shapes = []

for image in (images_dir.glob("*.png")):
    img = np.array(Image.open(image))
    image_shapes.append(img.shape[:2])  # (h, w)

for mask in masks_dir.glob("*.png"):
    m = np.array(Image.open(mask))
    mask_shapes.append(m.shape[:2])  # (h, w)

# check consistency
unique_image_shapes = set(image_shapes)
unique_mask_shapes = set(mask_shapes)

if len(unique_image_shapes) > 1:
    print("⚠️ Not all images have the same size:", unique_image_shapes)

if len(unique_mask_shapes) > 1:
    print("⚠️ Not all masks have the same size:", unique_mask_shapes)

if unique_image_shapes != unique_mask_shapes:
    print("⚠️ Image and mask sizes differ.")
else:
    print("✅ All images and masks have the same size:", unique_image_shapes.pop())

✅ All images and masks have the same size: (648, 864)


In [54]:
for sub in ["images", "masks"]:
    jpg_files = glob.glob(str(masks_dir / "*.jpg")) + glob.glob(str(masks_dir / "*.JPG"))
    for f in jpg_files:
        path = Path(f)
        try:
            path.unlink()  # delete the file
            print(f"🗑️ Removed {path}")
        except Exception as e:
            print(f"⚠️ Could not remove {path}: {e}")

🗑️ Removed /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset/masks/189.jpg
🗑️ Removed /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset/masks/162.jpg
🗑️ Removed /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset/masks/176.jpg
🗑️ Removed /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset/masks/228.jpg
🗑️ Removed /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset/masks/214.jpg
🗑️ Removed /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset/masks/200.jpg
🗑️ Removed /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset/masks/201.jpg
🗑️ Removed /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset/masks/215.jpg
🗑️ Removed /Users/ratchanonkhong

In [52]:
img_set  = sorted(images_dir.glob("*.png"))
mask_set = sorted(masks_dir.glob("*.png"))

# build dictionary: {"1": Path("masks/1.png"), "2": Path("masks/2.png"), ...}
mask_dict = {m.stem: m for m in mask_set}

# pair image with its mask if stem matches
pairs = [(p, mask_dict[p.stem]) for p in img_set if p.stem in mask_dict]

print(f"Found {len(pairs)} matched pairs.")
img_stems  = {p.stem for p in images_dir.glob("*.png")}
mask_stems = {p.stem for p in masks_dir.glob("*.png")}

# Images without masks
missing_masks = img_stems - mask_stems
print(f"🟥 {len(missing_masks)} images without masks:")
for s in sorted(missing_masks):
    print(" -", s)
    
missing_images = mask_stems - img_stems
print(f"🟧 {len(missing_images)} masks without images:")
for s in sorted(missing_images):
    print(" -", s)

Found 103 matched pairs.
🟥 197 images without masks:
 - 105
 - 106
 - 107
 - 108
 - 109
 - 110
 - 111
 - 112
 - 113
 - 114
 - 115
 - 116
 - 117
 - 118
 - 119
 - 120
 - 121
 - 122
 - 123
 - 124
 - 125
 - 126
 - 127
 - 128
 - 129
 - 130
 - 131
 - 132
 - 133
 - 134
 - 135
 - 136
 - 137
 - 138
 - 139
 - 140
 - 141
 - 142
 - 143
 - 144
 - 145
 - 146
 - 147
 - 148
 - 149
 - 150
 - 151
 - 152
 - 153
 - 154
 - 155
 - 156
 - 157
 - 158
 - 159
 - 160
 - 161
 - 162
 - 163
 - 164
 - 165
 - 166
 - 167
 - 168
 - 169
 - 170
 - 171
 - 172
 - 173
 - 174
 - 175
 - 176
 - 177
 - 178
 - 179
 - 180
 - 181
 - 182
 - 183
 - 184
 - 185
 - 186
 - 187
 - 188
 - 189
 - 190
 - 191
 - 192
 - 193
 - 194
 - 195
 - 196
 - 197
 - 198
 - 199
 - 200
 - 201
 - 202
 - 203
 - 204
 - 205
 - 206
 - 207
 - 208
 - 209
 - 210
 - 211
 - 212
 - 213
 - 214
 - 215
 - 216
 - 217
 - 218
 - 219
 - 220
 - 221
 - 222
 - 223
 - 224
 - 225
 - 226
 - 227
 - 228
 - 229
 - 230
 - 231
 - 232
 - 233
 - 234
 - 235
 - 236
 - 237
 - 238
 - 239
 -

In [51]:
train_imgs, val_imgs, train_masks, val_masks = train_test_split(
    images_dir, masks_dir, test_size=0.2, random_state=SEED
)

TypeError: Input should have at least 1 dimension i.e. satisfy `len(x.shape) > 0`, got scalar `array(PosixPath('/Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset/images'),
      dtype=object)` instead.

In [ ]:
pad_to_32 = A.PadIfNeeded(
    min_height=None, min_width=None,
    pad_height_divisor=32, pad_width_divisor=32,
    border_mode=cv2.BORDER_CONSTANT,  # pad with black for image, 0 for mask
    value=0, mask_value=0
)

train_transform = A.Compose([
    pad_to_32,
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    pad_to_32,
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])